In [9]:
import torch
import esm
from model import SparseForTokenClassification
from typing import Tuple, List
from pair_ranking_script import load_fasta_as_tuples

In [10]:
_, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()

In [11]:
data1 = load_fasta_as_tuples("NZ_ANAO01000007_products_concatenated_pros.fasta")
data2 = load_fasta_as_tuples("NZ_ANAO01000007_products_proteins.fasta")
print(len(data1[0][1]))

51986


In [12]:
batch_labels1, batch_strs1, batch_tokens1 = batch_converter(data1) # GENOME
batch_labels2, batch_strs2, batch_tokens2 = batch_converter(data2) # PROTEINS
print(batch_tokens1.shape)
print(batch_tokens2.shape)

torch.Size([1, 51988])
torch.Size([152, 1072])


In [13]:
version = '3C' # or '5B'
#checkpoint = torch.load(f'./models/{version}/config_and_model.pth', map_location='cpu', weights_only=False)
checkpoint = torch.load(f'/home/thibaut/mahdi/models/{version}/config_and_model.pth', map_location='cpu', weights_only=False)

In [14]:
device = "cuda:1" # Change if needed for availabel gpu
config = checkpoint['config']
model_state_dict = checkpoint['model_state_dict']
sparse_model = SparseForTokenClassification(config=config)
sparse_model.load_state_dict(model_state_dict)
sparse_model = sparse_model.to(device)
sparse_model = sparse_model.eval()

In [16]:
with torch.no_grad():
    batch_tokens1 = batch_tokens1.to(device)
    prots_lengths = [len(prot[1]) for prot in data2]
    print(prots_lengths)
    output2 = sparse_model(input_ids=batch_tokens1, output_attentions = True, two_step_selection= True, proteins_sizes = torch.tensor(prots_lengths))

# Very long (approx 3h for 52k genome) - due to import number of pairs 
# Returns the dictionnary of protein pairs with score

[1014, 174, 306, 296, 299, 85, 240, 466, 388, 374, 492, 490, 65, 391, 236, 471, 291, 644, 200, 103, 609, 85, 145, 142, 66, 430, 463, 515, 573, 433, 461, 210, 157, 72, 420, 314, 270, 67, 383, 437, 390, 338, 283, 114, 350, 390, 113, 799, 215, 308, 311, 494, 131, 298, 330, 537, 152, 449, 339, 308, 322, 301, 593, 41, 140, 483, 416, 225, 463, 173, 326, 477, 185, 144, 125, 475, 214, 218, 276, 242, 486, 323, 228, 490, 477, 432, 128, 392, 338, 420, 90, 172, 182, 566, 43, 423, 487, 788, 125, 471, 377, 623, 160, 978, 268, 168, 40, 223, 487, 170, 466, 308, 174, 399, 448, 235, 534, 660, 238, 273, 416, 223, 234, 458, 273, 310, 365, 188, 289, 167, 84, 434, 230, 534, 658, 526, 182, 965, 516, 288, 1070, 391, 631, 689, 162, 295, 380, 217, 258, 153, 239, 106]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.88 GiB. GPU 1 has a total capacity of 47.53 GiB of which 1.64 GiB is free. Process 2828943 has 14.95 GiB memory in use. Process 2882010 has 15.21 GiB memory in use. Including non-PyTorch memory, this process has 15.70 GiB memory in use. Of the allocated memory 12.69 GiB is allocated by PyTorch, and 2.70 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Here OOM due to other process on GPU. 